End to End model built on top of bert to classify sentences

In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
import numpy as np

In [3]:
from official.nlp import optimization  # to create AdamW optimizer

c:\Users\siddh\anaconda3\envs\project\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\siddh\anaconda3\envs\project\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
I

In [4]:
import tensorflow_text as text

In [5]:
# Load the text data from the file
with open('combined.txt', 'r', encoding='utf-8') as file:
    text_data = file.readlines()

# Create labels (half 0s and half 1s)
labels = [1] * (len(text_data) // 2) + [0] * (len(text_data) // 2)

# Combine text and labels
data = list(zip(text_data, labels))

In [6]:
labels[4000]

0

In [7]:
import random

In [8]:
# Load the text data from the file
with open('1_combined.txt', 'r', encoding='utf-8') as file:
    text_data = file.readlines()

# Create labels (half 0s and half 1s)
labels = [1] * (len(text_data) // 2) + [0] * (len(text_data) // 2)

# Combine text and labels
data = list(zip(text_data, labels))

# Shuffle the combined data
random.seed(42)  # For reproducibility
random.shuffle(data)

# Split the combined data into training, validation, and test sets
num_samples = len(data)
num_train = int(0.6 * num_samples)
num_val = int(0.2 * num_samples)

train_data = data[:num_train]
val_data = data[num_train:num_train+num_val]
test_data = data[num_train+num_val:]

# Create TensorFlow datasets
batch_size = 32

def text_label_generator(data):
    for text, label in data:
        yield text, label

train_ds = tf.data.Dataset.from_generator(
    lambda: text_label_generator(train_data),
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
train_ds = train_ds.shuffle(buffer_size=num_train)
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    lambda: text_label_generator(val_data),
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    lambda: text_label_generator(test_data),
    output_signature=(
        tf.TensorSpec(shape=(), dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
)
test_ds = test_ds.batch(batch_size)

class_names = ['Not', 'Idiom']  # Labels for 0 and 1

for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        sentence = text_batch[i].numpy().decode('utf-8')  # Decoding the bytes to string
        label = label_batch[i].numpy()
        print(f'Sentence: {sentence}')
        print(f'Label: {label} ({class_names[label]})')

Sentence: One can also say that with his third championship he had thrown in the towel .

Label: 1 (Idiom)
Sentence: They had just simply and totally fallen out of love with each other , and like the devout Catholics they were , carried on as a pretence .

Label: 1 (Idiom)
Sentence: Large-scale , publicly-owned enterprises will die and wither away well before the state which spawned them .

Label: 0 (Not)


In [9]:
# Define the URL for the BERT model
bert_model_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"

# Load the BERT model
bert_model = hub.KerasLayer(bert_model_url)

In [10]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1"
# Load the BERT preprocessing module
bert_preprocess = hub.load(preprocess_url)

In [11]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_word_ids', 'input_type_ids', 'input_mask']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [12]:
bert_results = bert_model(text_preprocessed)

print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.7626286   0.9928098  -0.18611893  0.36673865  0.15233758  0.65504503
  0.9681154  -0.9486271   0.00216119 -0.9877731   0.06842748 -0.97630596]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.28946334  0.3432125   0.33231455 ...  0.21300778  0.710208
  -0.05771144]
 [-0.28741926  0.31981003 -0.23018411 ...  0.58455145 -0.21329735
   0.7269199 ]
 [-0.6615691   0.68876886 -0.8743307  ...  0.10877214 -0.26173237
   0.4785539 ]
 ...
 [-0.22561145 -0.28925598 -0.07064315 ...  0.4756602   0.832772
   0.40025413]
 [-0.29824206 -0.2747315  -0.05450486 ...  0.48849693  1.0955367
   0.18163429]
 [-0.44378254  0.00930669  0.07223734 ...  0.17290081  1.1833241
   0.07898083]]


In [13]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(preprocess_url, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(bert_model_url, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [14]:
classifier_model = build_classifier_model()
bert_raw_result = classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

tf.Tensor([[0.41991055]], shape=(1, 1), dtype=float32)


In [15]:
tf.keras.utils.plot_model(classifier_model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [16]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [17]:
epochs = 10
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [18]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [19]:
print(f'Training model with {bert_model_url}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/10


118/118 [==============================] - 439s 4s/step - loss: 0.7764 - binary_accuracy: 0.4953 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 2/10
118/118 [==============================] - 412s 3s/step - loss: 0.7792 - binary_accuracy: 0.4983 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 3/10
118/118 [==============================] - 407s 3s/step - loss: 0.7771 - binary_accuracy: 0.4930 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 4/10
118/118 [==============================] - 410s 3s/step - loss: 0.7862 - binary_accuracy: 0.4898 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 5/10
118/118 [==============================] - 406s 3s/step - loss: 0.7742 - binary_accuracy: 0.4943 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 6/10
118/118 [==============================] - 406s 3s/step - loss: 0.7819 - binary_accuracy: 0.4935 - val_loss: 0.7449 - val_binary_accuracy: 0.5120
Epoch 7/10
118/118 [==============================] - 416s 4s/step - 

KeyboardInterrupt: 

In [20]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

40/40 [==============================] - 44s 1s/step - loss: 0.7477 - binary_accuracy: 0.5004
Loss: 0.7477088570594788
Accuracy: 0.5003983974456787


In [ ]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i][0]:.6f}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()

In [ ]:
examples = [
    'ill be keeping an eye on it',  # this is the same sentence tried earlier
    'dramatic advance in microscopy',
    'dont beat around the bush',
]

original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

In [ ]:
print_my_examples(examples, original_results)

input: ill be keeping an eye on it    : score: 0.334516
input: dramatic advance in microscopy : score: 0.246825
input: dont beat around the bush      : score: 0.283206

